<div style="background-color: #add8e6; padding: 10px; height: 70px; border-radius: 15px;">
    <div style="font-family: 'Georgia', serif; font-size: 20px; padding: 10px; text-align: right; position: absolute; right: 20px;">
        Mohammad Idrees Bhat <br>
        <span style="font-family: 'Arial', sans-serif;font-size: 12px; color: #0a0a0a;">Tech Skills Trainer | AI/ML Consultant</span> <!--- Mohammad Idrees Bhat | Tech Skills Trainer | AI/ML Consultant --->
    </div>
</div>

<!--- Mohammad Idrees Bhat | Tech Skills Trainer | AI/ML Consultant --->

<div style="background-color: #002147; padding: 10px; text-align: center; color: white; font-size: 32px; font-family: 'Arial', sans-serif;">
    Apache Tools for Big Data <br>
    <h3 style="text-align: center; color: white; font-size: 15px; font-family: 'Arial', sans-serif;">Apache Airflow, Apache Hadoop, Apache Spark</h3>
</div>

<div style="background-color: white; color: black; padding: 10px;">
    <h4><b>AGENDA</b> <p><p>
1.  <p><p> 
2.   <p>
3.   <p>
4.   <p>  
</h4> </div>

<!-- Link the Montserrat font -->
<link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@700&display=swap" rel="stylesheet">

<!-- Main div with centered content and a flexible box size, no scroll bar -->
<div style="background-color: #baf733; min-height: 100px; width: 100%; display: flex; justify-content: center; align-items: center; position: relative; padding: 20px; box-sizing: border-box; font-family: 'Montserrat', sans-serif; font-weight: 700; font-size: 20px; border-radius: 15px;">
    <div style="position: absolute; top: 10px; right: 10px; padding: 5px 10px; font-size: 14px; color: rgba(0, 0, 0, 0.05); border-radius: 10px;">Mohammad Idrees Bhat</div>
    <!-- Fill the below text with question -->
    <!-- Fill the below text with question -->
    In all that you do, be kind!
    <!-- Fill the above text with question -->
    <!-- Fill the above text with question -->

</div>

<div style="background-color: lightgreen; color: black; padding: 4px;">
    <h3>1. topic 1 
</h3> </div>

<div style="background-color: lightblue; color: black; padding: 4px;">
    <h4> sub heading 1
</h4> </div>

Extraction

we will scrape amazon books, 

clean and transformed data

is loaded

stored in data warehouse

- Airflow will help us schedule tasks
    (docker with airflow)
    - scheduler
    - webserver for ui
    - 


- Python as primary language, and SQL for querries



Step 1 is to create and activate a new virtual environment

- Download python 3.8 at some place, most likely the path will look like this: `C:\Users\devid\AppData\Local\Programs\Python\Python38`
- Use this command to create the environment:
`C:\Users\<username>\AppData\Local\Programs\Python\Python38\python.exe -m venv <environment_name>`
- In the terminal, (a cmd terminal in VS code, not powershell) use this command:
`<environment_name>\Scripts\activate`


Step 2: Install docker and airflow

**1.** Go to this page:
https://airflow.apache.org/docs/apache-airflow/stable/howto/docker-compose/index.html
- Ensure that docker compose is installed:
`docker compose version`
- If it is not installed, use this: 
[Docker Windows](https://docs.docker.com/desktop/setup/install/windows-install/) or [Docker Mac](https://docs.docker.com/desktop/setup/install/mac-install/)
**2.** Copy the **Fetching docker-compose.yaml** command and paste in the activated terminal in Linux/Mac:

    `curl -LfO 'https://airflow.apache.org/docs/apache-airflow/2.10.3/docker-compose.yaml'`

- But use double quotes on windows:

    `curl -LfO "https://airflow.apache.org/docs/apache-airflow/2.10.3/docker-compose.yaml"`

**3.** Then use this on Linux/Mac:
    ```bash
    mkdir -p ./dags ./logs ./plugins ./config
    echo -e "AIRFLOW_UID=$(id -u)" > .env
    ```

- But use this on windows:
    ```bash
    mkdir dags logs plugins config 
    echo AIRFLOW_UID=50000 > .env
    ```
These commands create the folders dags, logs, plugins, config and another `.env` file


The next step will require docker to be present on your system, in your environment, and running to execute the command.

**4.** Initialilze database, use this command:
    `docker compose up airflow-init`

**5.** Start all services using:
    `docker compose up`

Then we can visit `localhost:8080` from our browser to see the airflow interface

<div style="background-color: lightblue; color: black; padding: 4px;">
    <h4> sub heading 1
</h4> </div>

- Add new port to the `docker-compose.yaml` file within `services:`

```yml
    ports:
      -  "5432:5432"
```

- Next add one more port:

```yml
  pgadmin:
      container_name: pgadmin4_container2   
      image: dpage/pgadmin4  
      restart: always 
      environment:
        PGADMIN_DEFAULT_EMAIL: admin@admin.com
        PGADMIN_DEFAULT_PASSWORD: root
      ports:
        -  "5050:80"
```
- Then open a new command prompt and close the docker container using:
`docker compose down`

- Then start the docker container again using:
`docker compose up`

- Go to localhost:5050 and login to pgadmin, with the username password setup earlier. 

<div style="background-color: lightblue; color: black; padding: 4px;">
    <h4> sub heading 1
</h4> </div>

![pipeline_design.png](pipeline_design.png)

Now it's your turn!
### Task 1: description of task

    - instructions

<div style="background-color: lightblue; color: white; padding: 10px; text-align: center;">
    <h1>_________________________________END________________________________
        <!--- Mohammad Idrees Bhat | Tech Skills Trainer | AI/ML Consultant --->
</h1> </div>

<div style="background-color: #002147; color: #fff; padding: 30px; text-align: center;">
    <h1>THANK YOU!
        <!--- Mohammad Idrees Bhat | Tech Skills Trainer | AI/ML Consultant --->
</h1> </div>

<div style="background-color: lightgreen; color: black; padding: 30px;">
    <h4> Live Exercise Solutions
        
</h4> </div>

In [ ]:
solutions

<div class="alert alert-block alert-warning"  padding: 10px; text-align: center;">
    <font size="3"> Programming Interveiw Questions</font>
</div>

1. topic:
    - question

<!--- Mohammad Idrees Bhat | Mohammad Idrees Bhat --->

<h2 style="background-color: #ffe4e1; color: #2f4f4f; padding: 10px; border-radius: 10px; width: 350px; text-align: center; float: right; margin: 20px 0;">
    Mohammad Idrees Bhat<br>
    <span style="font-size: 12px; color: #696969;">
        Tech Skills Trainer | AI/ML Consultant
    </span>
</h2>

<!--- Mohammad Idrees Bhat | Tech Skills Trainer | AI/ML Consultant --->